In [3]:
#Importing packages
#
# You can delete whatever package(s) you want..
#
import json
import time
import requests
import math as m
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import descartes
import geopandas as gpd
from shapely.geometry import Point, Polygon
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split 
from sklearn.linear_model import LinearRegression
from sklearn import metrics
from scipy.optimize import curve_fit
import plotly.express as px
import chart_studio
import chart_studio.plotly as py
import chart_studio.tools as tls
import folium



# Leave This code here and lookup at pandas documentation 
# if you need to know about chained assignments
pd.options.mode.chained_assignment = None  # default='warn'

# Because most of my data sets I have made can have up to 30+ columns and 20+ rows
# code below will increase pandas defaults for max rows 
# and columns that you can display in a Juptyer Notebook
pd.options.display.max_columns = 60
pd.options.display.max_rows = 1000

## IP STACK API

- [IP Stack](https://ipstack.com/documentation)

In [4]:
# Get access token from file
with open('access_key.txt', 'r') as file:
    access_key = file.read().replace('\n', '')
access_key = access_key.split('=')[1]

# # Get the data from the API
url = 'http://api.ipstack.com/100.7.78.240?&fields=country_code,country_name,city,latitude,longitude'
payload = {'access_key': '{key}'.format(key=access_key)}
reqs = requests.get(url, params=payload).json()
reqs

{'longitude': -77.63590240478516,
 'latitude': 37.5341682434082,
 'city': 'Bon Air',
 'country_name': 'United States',
 'country_code': 'US'}

## Building the Pandas Dataframe for our Relay Data

In [5]:
df = pd.read_json('relays.json')
df2 = pd.read_csv('ip_hosts.csv')

In [7]:
# Let's make a few copies of the data frame to test our data collection
df_copy = df.copy()
df_ip_hosts = df2.copy()
df_copy

,dns,dns_srv,ipv4,ipv6,port,live_stake,delegators,description,hash,hex,homepage,name,pool_id,ticker,url
0,None,None,54.220.20.40,None,3002,49569650567271,3340,Octa's Performance Pool,ca7d12decf886e31f5226b5946c62edc81a7e40af95ce7...,153806dbcd134ddee69a8c5204e38ac80448f62342f8c2...,https://octaluso.dyndns.org,OctasPool,pool1z5uqdk7dzdxaae5633fqfcu2eqzy3a3rgtuvy087f...,OCTAS,https://raw.githubusercontent.com/Octalus/card...
1,relays.mainnet.stakenuts.com,None,None,None,3001,19157905491154,165,StakeNuts.com,47c0c68cb57f4a5b4a87bad896fc274678e7aea98e200f...,0f292fcaa02b8b2f9b3c8f9fd8e0bb21abedb692a6d505...,https://stakenuts.com/,StakeNuts,pool1pu5jlj4q9w9jlxeu370a3c9myx47md5j5m2str0na...,NUTS,https://stakenuts.com/mainnet.json
2,relay.zenithpool.io,None,None,None,31400,547697202732,9,"One owner, one operator, one pool. Operating s...",21cc44498ab6e74e081a805e629fff171a66d0c8d034f9...,c1ede3cc9133209466774d4826044e408db13d6fe6df75...,https://zenithpool.io,ZENITH,pool1c8k78ny3xvsfgenhf4yzvpzwgzxmz0t0um0h2xnn2...,ZEN,https://metadata.zenithpool.io
3,None,None,95.183.53.20,None,6060,1862316910352,707,"Stability, Security, Reliability, Neutrality! ...",34fdde237812fab14d29a80423bb295f39122f4fea1aae...,01df29429173d263c7533a22742dae19f16a08798b7a57...,https://ispool.live/,Switzerland Investment,pool1q80jjs53w0fx836n8g38gtdwr8ck5zre3da90peux...,000,https://ispool.live/metadata
4,None,None,95.217.222.194,None,6060,1862316910352,707,"Stability, Security, Reliability, Neutrality! ...",34fdde237812fab14d29a80423bb295f39122f4fea1aae...,01df29429173d263c7533a22742dae19f16a08798b7a57...,https://ispool.live/,Switzerland Investment,pool1q80jjs53w0fx836n8g38gtdwr8ck5zre3da90peux...,000,https://ispool.live/metadata
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5413,None,None,137.184.178.161,None,6000,130027614,1,"midnort staking pool, your nordic Cardano Stak...",cb738f574938eaa9e960e052f071bfb62e19950e1f3acf...,09a0c14484d3f421e813bfd509b102be1d327aa1077aae...,https://cryptoginger29.wixsite.com/midnort,midnort-ada,pool1pxsvz3yy606zr6qnhl2snvgzhcwny74pqaa2uvrk7...,MIDN,https://git.io/JKN8b
5414,None,None,46.101.17.79,None,6000,130027614,1,"midnort staking pool, your nordic Cardano Stak...",cb738f574938eaa9e960e052f071bfb62e19950e1f3acf...,09a0c14484d3f421e813bfd509b102be1d327aa1077aae...,https://cryptoginger29.wixsite.com/midnort,midnort-ada,pool1pxsvz3yy606zr6qnhl2snvgzhcwny74pqaa2uvrk7...,MIDN,https://git.io/JKN8b
5415,None,None,72.86.45.91,None,3001,18295901333,1,None,92cf348da2c6a2a0666196009b4eb97101d3231955fd84...,b790c017d56d28406215468c4355c4525bf497c14a5c8c...,None,None,pool1k7gvq974d55yqcs4g6xyx4wy2fdlf97pffwgesjfq...,None,https://git.io/J6rex
5416,r1.jaguares.org,None,None,None,3001,10008229492,1,None,0915cb165e3532496e7183fce748827ac1f1aceebec819...,70a96f885282fe2d810ca500713ba64998491257054bc2...,None,None,pool1wz5klzzjstlzmqgv55q8zwaxfxvyjyjhq49u9hg3f...,None,https://git.io/J62wR


In [8]:
# We need to replace the NaN values with empty strings
df_copy.fillna('', inplace=True)


In [29]:
df_copy

,dns,dns_srv,ipv4,ipv6,port,live_stake,delegators,description,hash,hex,homepage,name,pool_id,ticker,url
0,,,54.220.20.40,,3002,49569650567271,3340,Octa's Performance Pool,ca7d12decf886e31f5226b5946c62edc81a7e40af95ce7...,153806dbcd134ddee69a8c5204e38ac80448f62342f8c2...,https://octaluso.dyndns.org,OctasPool,pool1z5uqdk7dzdxaae5633fqfcu2eqzy3a3rgtuvy087f...,OCTAS,https://raw.githubusercontent.com/Octalus/card...
1,relays.mainnet.stakenuts.com,,,,3001,19157905491154,165,StakeNuts.com,47c0c68cb57f4a5b4a87bad896fc274678e7aea98e200f...,0f292fcaa02b8b2f9b3c8f9fd8e0bb21abedb692a6d505...,https://stakenuts.com/,StakeNuts,pool1pu5jlj4q9w9jlxeu370a3c9myx47md5j5m2str0na...,NUTS,https://stakenuts.com/mainnet.json
2,relay.zenithpool.io,,,,31400,547697202732,9,"One owner, one operator, one pool. Operating s...",21cc44498ab6e74e081a805e629fff171a66d0c8d034f9...,c1ede3cc9133209466774d4826044e408db13d6fe6df75...,https://zenithpool.io,ZENITH,pool1c8k78ny3xvsfgenhf4yzvpzwgzxmz0t0um0h2xnn2...,ZEN,https://metadata.zenithpool.io
3,,,95.183.53.20,,6060,1862316910352,707,"Stability, Security, Reliability, Neutrality! ...",34fdde237812fab14d29a80423bb295f39122f4fea1aae...,01df29429173d263c7533a22742dae19f16a08798b7a57...,https://ispool.live/,Switzerland Investment,pool1q80jjs53w0fx836n8g38gtdwr8ck5zre3da90peux...,000,https://ispool.live/metadata
4,,,95.217.222.194,,6060,1862316910352,707,"Stability, Security, Reliability, Neutrality! ...",34fdde237812fab14d29a80423bb295f39122f4fea1aae...,01df29429173d263c7533a22742dae19f16a08798b7a57...,https://ispool.live/,Switzerland Investment,pool1q80jjs53w0fx836n8g38gtdwr8ck5zre3da90peux...,000,https://ispool.live/metadata
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5413,,,137.184.178.161,,6000,130027614,1,"midnort staking pool, your nordic Cardano Stak...",cb738f574938eaa9e960e052f071bfb62e19950e1f3acf...,09a0c14484d3f421e813bfd509b102be1d327aa1077aae...,https://cryptoginger29.wixsite.com/midnort,midnort-ada,pool1pxsvz3yy606zr6qnhl2snvgzhcwny74pqaa2uvrk7...,MIDN,https://git.io/JKN8b
5414,,,46.101.17.79,,6000,130027614,1,"midnort staking pool, your nordic Cardano Stak...",cb738f574938eaa9e960e052f071bfb62e19950e1f3acf...,09a0c14484d3f421e813bfd509b102be1d327aa1077aae...,https://cryptoginger29.wixsite.com/midnort,midnort-ada,pool1pxsvz3yy606zr6qnhl2snvgzhcwny74pqaa2uvrk7...,MIDN,https://git.io/JKN8b
5415,,,72.86.45.91,,3001,18295901333,1,,92cf348da2c6a2a0666196009b4eb97101d3231955fd84...,b790c017d56d28406215468c4355c4525bf497c14a5c8c...,,,pool1k7gvq974d55yqcs4g6xyx4wy2fdlf97pffwgesjfq...,,https://git.io/J6rex
5416,r1.jaguares.org,,,,3001,10008229492,1,,0915cb165e3532496e7183fce748827ac1f1aceebec819...,70a96f885282fe2d810ca500713ba64998491257054bc2...,,,pool1wz5klzzjstlzmqgv55q8zwaxfxvyjyjhq49u9hg3f...,,https://git.io/J62wR


In [31]:
df_copy.iloc[3888,0]

'None'

## Build a function to return a csv and Excel file containing the IP hosts for every relay

In [43]:


def get_ip_host(df, outputfileName):
	
	ip_host_lst = []

	if type(df) != pd.DataFrame:
		raise TypeError('df must be a pandas dataframe')
	if type(outputfileName) != str:
		raise TypeError('outputfileName must be a string')


	for i in range(len(df)):
		# Raise type error if the data is not a string
		if type(df['dns'][i]) != str:
			print(df['dns'][i])
			raise TypeError('dns must be a string')

		if type(df['dns_srv'][i]) != str:
			print(df['dns_srv'][i])
			raise TypeError('dns_srv must be a string')

		if type(df['ipv4'][i]) != str:
			print(df['ipv4'][i])
			raise TypeError('ipv4 must be a string')

		if type(df['ipv6'][i]) != str:	
			print(df['ipv6'][i])
			raise TypeError('ipv6 must be a string')

		# This code maps the ip host to its pool_id as a tuple
		if len(df['dns'][i]) > 0 or len(df['dns'][i]) > 4:
			ip_host_lst.append(tuple((df['dns'][i], df['port'][i], df['pool_id'][i],
				df['live_stake'][i], df['delegators'][i], df['description'][i],
				df['hash'][i], df['hex'][i], df['homepage'][i], df['name'][i],	
				df['ticker'][i], df['url'][i])))

		if len(df['dns_srv'][i]) > 0 or len(df['dns_srv'][i]) > 4:
			ip_host_lst.append(tuple((df['dns_srv'][i], df['port'][i], df['pool_id'][i],
			df['live_stake'][i], df['delegators'][i], df['description'][i],
			df['hash'][i], df['hex'][i], df['homepage'][i], df['name'][i],
			df['ticker'][i], df['url'][i])))

		if len(df['ipv4'][i]) > 0 or len(df['ipv4'][i]) > 4:
			ip_host_lst.append(tuple((df['ipv4'][i], df['port'][i], df['pool_id'][i],
			df['live_stake'][i], df['delegators'][i], df['description'][i],
			df['hash'][i], df['hex'][i], df['homepage'][i], df['name'][i],
			df['ticker'][i], df['url'][i])))

		if len(df['ipv6'][i]) > 0 or len(df['ipv6'][i]) > 4:
			ip_host_lst.append(tuple((df['ipv6'][i], df['port'][i], df['pool_id'][i],
			df['live_stake'][i], df['delegators'][i], df['description'][i],
			df['hash'][i], df['hex'][i], df['homepage'][i], df['name'][i],
			df['ticker'][i], df['url'][i])))
		

		
		

	if len(ip_host_lst) == 0:
		print('No IPs found')

	if len(ip_host_lst) != len(df):
		print('Number of rows in host lst: '+str(len(ip_host_lst)), 'Number of Rows in original df: '+str(len(df)), sep='\n')
	
	ip_host_df = pd.DataFrame(pd.Series(ip_host_lst))
	ip_host_df.columns = ['ip_host']

	
	return ip_host_df.to_csv(r''+outputfileName+'.csv', index=False)

## Let's go ahead and attempt to resolve the DNS names for the IP addresses

In [44]:
get_ip_host(df_copy, 'all_relays')



Number of rows in host lst: 5509
Number of Rows in original df: 5418


In [45]:
testing = pd.read_csv('all_relays.csv')
testing

,ip_host
0,"('54.220.20.40', 3002.0, 'pool1z5uqdk7dzdxaae5..."
1,"('relays.mainnet.stakenuts.com', 3001.0, 'pool..."
2,"('relay.zenithpool.io', 31400.0, 'pool1c8k78ny..."
3,"('95.183.53.20', 6060.0, 'pool1q80jjs53w0fx836..."
4,"('95.217.222.194', 6060.0, 'pool1q80jjs53w0fx8..."
...,...
5504,"('137.184.178.161', 6000, 'pool1pxsvz3yy606zr6..."
5505,"('46.101.17.79', 6000, 'pool1pxsvz3yy606zr6qnh..."
5506,"('72.86.45.91', 3001, 'pool1k7gvq974d55yqcs4g6..."
5507,"('r1.jaguares.org', 3001, 'pool1wz5klzzjstlzmq..."


In [46]:
# check type of data
type(testing['ip_host'][0])

str

In [47]:
# Need to convert the all_relays.csv data to a tuple object from the string
# we will use literal_eval to convert the string to a tuple

import ast

testing['ip_host_tuple'] = testing['ip_host'].apply(ast.literal_eval)
testing

,ip_host,ip_host_tuple
0,"('54.220.20.40', 3002.0, 'pool1z5uqdk7dzdxaae5...","(54.220.20.40, 3002.0, pool1z5uqdk7dzdxaae5633..."
1,"('relays.mainnet.stakenuts.com', 3001.0, 'pool...","(relays.mainnet.stakenuts.com, 3001.0, pool1pu..."
2,"('relay.zenithpool.io', 31400.0, 'pool1c8k78ny...","(relay.zenithpool.io, 31400.0, pool1c8k78ny3xv..."
3,"('95.183.53.20', 6060.0, 'pool1q80jjs53w0fx836...","(95.183.53.20, 6060.0, pool1q80jjs53w0fx836n8g..."
4,"('95.217.222.194', 6060.0, 'pool1q80jjs53w0fx8...","(95.217.222.194, 6060.0, pool1q80jjs53w0fx836n..."
...,...,...
5504,"('137.184.178.161', 6000, 'pool1pxsvz3yy606zr6...","(137.184.178.161, 6000, pool1pxsvz3yy606zr6qnh..."
5505,"('46.101.17.79', 6000, 'pool1pxsvz3yy606zr6qnh...","(46.101.17.79, 6000, pool1pxsvz3yy606zr6qnhl2s..."
5506,"('72.86.45.91', 3001, 'pool1k7gvq974d55yqcs4g6...","(72.86.45.91, 3001, pool1k7gvq974d55yqcs4g6xyx..."
5507,"('r1.jaguares.org', 3001, 'pool1wz5klzzjstlzmq...","(r1.jaguares.org, 3001, pool1wz5klzzjstlzmqgv5..."


In [48]:
# Check the tuples column to make sure the data type is correct
type(testing['ip_host_tuple'][0])

tuple

In [49]:
# Let's make a function to resolve hostnames to IP addresses
import dns.resolver
def resolve_hostname(hostname):
	try:
		ip = dns.resolver.resolve(hostname, 'A')
		return ip[0].to_text()
	except:
		return ''

In [50]:
new_df = pd.DataFrame()
# for x,y in testing['ip_host_tuple']:

# 	new_df['ip_host'].append(pd.Series(x), ignore_index=True)

In [51]:
len(testing['ip_host_tuple'][0])

12

In [52]:
new_df['ip_host'], new_df['port'], new_df['pool_id'] , new_df['live_stake'], new_df['delegators'], new_df['description'], new_df['hash'], new_df['hex'], new_df['homepage'], new_df['name'], new_df['ticker'], new_df['url'] = testing.ip_host_tuple.str
new_df


,ip_host,port,pool_id,live_stake,delegators,description,hash,hex,homepage,name,ticker,url
0,54.220.20.40,3002,pool1z5uqdk7dzdxaae5633fqfcu2eqzy3a3rgtuvy087f...,49569650567271,3340,Octa's Performance Pool,ca7d12decf886e31f5226b5946c62edc81a7e40af95ce7...,153806dbcd134ddee69a8c5204e38ac80448f62342f8c2...,https://octaluso.dyndns.org,OctasPool,OCTAS,https://raw.githubusercontent.com/Octalus/card...
1,relays.mainnet.stakenuts.com,3001,pool1pu5jlj4q9w9jlxeu370a3c9myx47md5j5m2str0na...,19157905491154,165,StakeNuts.com,47c0c68cb57f4a5b4a87bad896fc274678e7aea98e200f...,0f292fcaa02b8b2f9b3c8f9fd8e0bb21abedb692a6d505...,https://stakenuts.com/,StakeNuts,NUTS,https://stakenuts.com/mainnet.json
2,relay.zenithpool.io,31400,pool1c8k78ny3xvsfgenhf4yzvpzwgzxmz0t0um0h2xnn2...,547697202732,9,"One owner, one operator, one pool. Operating s...",21cc44498ab6e74e081a805e629fff171a66d0c8d034f9...,c1ede3cc9133209466774d4826044e408db13d6fe6df75...,https://zenithpool.io,ZENITH,ZEN,https://metadata.zenithpool.io
3,95.183.53.20,6060,pool1q80jjs53w0fx836n8g38gtdwr8ck5zre3da90peux...,1862316910352,707,"Stability, Security, Reliability, Neutrality! ...",34fdde237812fab14d29a80423bb295f39122f4fea1aae...,01df29429173d263c7533a22742dae19f16a08798b7a57...,https://ispool.live/,Switzerland Investment,000,https://ispool.live/metadata
4,95.217.222.194,6060,pool1q80jjs53w0fx836n8g38gtdwr8ck5zre3da90peux...,1862316910352,707,"Stability, Security, Reliability, Neutrality! ...",34fdde237812fab14d29a80423bb295f39122f4fea1aae...,01df29429173d263c7533a22742dae19f16a08798b7a57...,https://ispool.live/,Switzerland Investment,000,https://ispool.live/metadata
...,...,...,...,...,...,...,...,...,...,...,...,...
5504,137.184.178.161,6000,pool1pxsvz3yy606zr6qnhl2snvgzhcwny74pqaa2uvrk7...,130027614,1,"midnort staking pool, your nordic Cardano Stak...",cb738f574938eaa9e960e052f071bfb62e19950e1f3acf...,09a0c14484d3f421e813bfd509b102be1d327aa1077aae...,https://cryptoginger29.wixsite.com/midnort,midnort-ada,MIDN,https://git.io/JKN8b
5505,46.101.17.79,6000,pool1pxsvz3yy606zr6qnhl2snvgzhcwny74pqaa2uvrk7...,130027614,1,"midnort staking pool, your nordic Cardano Stak...",cb738f574938eaa9e960e052f071bfb62e19950e1f3acf...,09a0c14484d3f421e813bfd509b102be1d327aa1077aae...,https://cryptoginger29.wixsite.com/midnort,midnort-ada,MIDN,https://git.io/JKN8b
5506,72.86.45.91,3001,pool1k7gvq974d55yqcs4g6xyx4wy2fdlf97pffwgesjfq...,18295901333,1,,92cf348da2c6a2a0666196009b4eb97101d3231955fd84...,b790c017d56d28406215468c4355c4525bf497c14a5c8c...,,,,https://git.io/J6rex
5507,r1.jaguares.org,3001,pool1wz5klzzjstlzmqgv55q8zwaxfxvyjyjhq49u9hg3f...,10008229492,1,,0915cb165e3532496e7183fce748827ac1f1aceebec819...,70a96f885282fe2d810ca500713ba64998491257054bc2...,,,,https://git.io/J62wR


In [53]:
new_df['resolved_ip'] = new_df['ip_host'].apply(resolve_hostname)

In [54]:
# sprinkle some magic to get the resolved IPs and the IPs in ip_host into a single column
for i in range(len(new_df)):
	if len(new_df['resolved_ip'][i]) == 0:
		new_df['resolved_ip'][i] = new_df['ip_host'][i]

In [55]:
new_df

,ip_host,port,pool_id,live_stake,delegators,description,hash,hex,homepage,name,ticker,url,resolved_ip
0,54.220.20.40,3002,pool1z5uqdk7dzdxaae5633fqfcu2eqzy3a3rgtuvy087f...,49569650567271,3340,Octa's Performance Pool,ca7d12decf886e31f5226b5946c62edc81a7e40af95ce7...,153806dbcd134ddee69a8c5204e38ac80448f62342f8c2...,https://octaluso.dyndns.org,OctasPool,OCTAS,https://raw.githubusercontent.com/Octalus/card...,54.220.20.40
1,relays.mainnet.stakenuts.com,3001,pool1pu5jlj4q9w9jlxeu370a3c9myx47md5j5m2str0na...,19157905491154,165,StakeNuts.com,47c0c68cb57f4a5b4a87bad896fc274678e7aea98e200f...,0f292fcaa02b8b2f9b3c8f9fd8e0bb21abedb692a6d505...,https://stakenuts.com/,StakeNuts,NUTS,https://stakenuts.com/mainnet.json,199.247.23.219
2,relay.zenithpool.io,31400,pool1c8k78ny3xvsfgenhf4yzvpzwgzxmz0t0um0h2xnn2...,547697202732,9,"One owner, one operator, one pool. Operating s...",21cc44498ab6e74e081a805e629fff171a66d0c8d034f9...,c1ede3cc9133209466774d4826044e408db13d6fe6df75...,https://zenithpool.io,ZENITH,ZEN,https://metadata.zenithpool.io,193.122.12.254
3,95.183.53.20,6060,pool1q80jjs53w0fx836n8g38gtdwr8ck5zre3da90peux...,1862316910352,707,"Stability, Security, Reliability, Neutrality! ...",34fdde237812fab14d29a80423bb295f39122f4fea1aae...,01df29429173d263c7533a22742dae19f16a08798b7a57...,https://ispool.live/,Switzerland Investment,000,https://ispool.live/metadata,95.183.53.20
4,95.217.222.194,6060,pool1q80jjs53w0fx836n8g38gtdwr8ck5zre3da90peux...,1862316910352,707,"Stability, Security, Reliability, Neutrality! ...",34fdde237812fab14d29a80423bb295f39122f4fea1aae...,01df29429173d263c7533a22742dae19f16a08798b7a57...,https://ispool.live/,Switzerland Investment,000,https://ispool.live/metadata,95.217.222.194
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5504,137.184.178.161,6000,pool1pxsvz3yy606zr6qnhl2snvgzhcwny74pqaa2uvrk7...,130027614,1,"midnort staking pool, your nordic Cardano Stak...",cb738f574938eaa9e960e052f071bfb62e19950e1f3acf...,09a0c14484d3f421e813bfd509b102be1d327aa1077aae...,https://cryptoginger29.wixsite.com/midnort,midnort-ada,MIDN,https://git.io/JKN8b,137.184.178.161
5505,46.101.17.79,6000,pool1pxsvz3yy606zr6qnhl2snvgzhcwny74pqaa2uvrk7...,130027614,1,"midnort staking pool, your nordic Cardano Stak...",cb738f574938eaa9e960e052f071bfb62e19950e1f3acf...,09a0c14484d3f421e813bfd509b102be1d327aa1077aae...,https://cryptoginger29.wixsite.com/midnort,midnort-ada,MIDN,https://git.io/JKN8b,46.101.17.79
5506,72.86.45.91,3001,pool1k7gvq974d55yqcs4g6xyx4wy2fdlf97pffwgesjfq...,18295901333,1,,92cf348da2c6a2a0666196009b4eb97101d3231955fd84...,b790c017d56d28406215468c4355c4525bf497c14a5c8c...,,,,https://git.io/J6rex,72.86.45.91
5507,r1.jaguares.org,3001,pool1wz5klzzjstlzmqgv55q8zwaxfxvyjyjhq49u9hg3f...,10008229492,1,,0915cb165e3532496e7183fce748827ac1f1aceebec819...,70a96f885282fe2d810ca500713ba64998491257054bc2...,,,,https://git.io/J62wR,64.29.151.221


In [56]:
new_df.to_csv('resolved_hosts.csv', index=False)
new_df.to_json('resolved_hosts.json', orient='records')

In [13]:
# for i in new_df['ip_host']:
# 	try:
# 		hostname = dns.resolver.resolve(i, 'A')
# 		print(hostname[0].to_text())
# 	except:
# 		print(i)

# Don't modify below this line

In [57]:
# make the dataframe
# df_all_relays['ip_host'] = pd.read_json('all_relays.json')
# df_all_relays
hosts = pd.read_csv('resolved_hosts.csv')
# df_all_relays['ip host'] = hosts
# df_all_relays
hosts


,ip_host,port,pool_id,live_stake,delegators,description,hash,hex,homepage,name,ticker,url,resolved_ip
0,54.220.20.40,3002.0,pool1z5uqdk7dzdxaae5633fqfcu2eqzy3a3rgtuvy087f...,49569650567271,3340,Octa's Performance Pool,ca7d12decf886e31f5226b5946c62edc81a7e40af95ce7...,153806dbcd134ddee69a8c5204e38ac80448f62342f8c2...,https://octaluso.dyndns.org,OctasPool,OCTAS,https://raw.githubusercontent.com/Octalus/card...,54.220.20.40
1,relays.mainnet.stakenuts.com,3001.0,pool1pu5jlj4q9w9jlxeu370a3c9myx47md5j5m2str0na...,19157905491154,165,StakeNuts.com,47c0c68cb57f4a5b4a87bad896fc274678e7aea98e200f...,0f292fcaa02b8b2f9b3c8f9fd8e0bb21abedb692a6d505...,https://stakenuts.com/,StakeNuts,NUTS,https://stakenuts.com/mainnet.json,199.247.23.219
2,relay.zenithpool.io,31400.0,pool1c8k78ny3xvsfgenhf4yzvpzwgzxmz0t0um0h2xnn2...,547697202732,9,"One owner, one operator, one pool. Operating s...",21cc44498ab6e74e081a805e629fff171a66d0c8d034f9...,c1ede3cc9133209466774d4826044e408db13d6fe6df75...,https://zenithpool.io,ZENITH,ZEN,https://metadata.zenithpool.io,193.122.12.254
3,95.183.53.20,6060.0,pool1q80jjs53w0fx836n8g38gtdwr8ck5zre3da90peux...,1862316910352,707,"Stability, Security, Reliability, Neutrality! ...",34fdde237812fab14d29a80423bb295f39122f4fea1aae...,01df29429173d263c7533a22742dae19f16a08798b7a57...,https://ispool.live/,Switzerland Investment,000,https://ispool.live/metadata,95.183.53.20
4,95.217.222.194,6060.0,pool1q80jjs53w0fx836n8g38gtdwr8ck5zre3da90peux...,1862316910352,707,"Stability, Security, Reliability, Neutrality! ...",34fdde237812fab14d29a80423bb295f39122f4fea1aae...,01df29429173d263c7533a22742dae19f16a08798b7a57...,https://ispool.live/,Switzerland Investment,000,https://ispool.live/metadata,95.217.222.194
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5504,137.184.178.161,6000,pool1pxsvz3yy606zr6qnhl2snvgzhcwny74pqaa2uvrk7...,130027614,1,"midnort staking pool, your nordic Cardano Stak...",cb738f574938eaa9e960e052f071bfb62e19950e1f3acf...,09a0c14484d3f421e813bfd509b102be1d327aa1077aae...,https://cryptoginger29.wixsite.com/midnort,midnort-ada,MIDN,https://git.io/JKN8b,137.184.178.161
5505,46.101.17.79,6000,pool1pxsvz3yy606zr6qnhl2snvgzhcwny74pqaa2uvrk7...,130027614,1,"midnort staking pool, your nordic Cardano Stak...",cb738f574938eaa9e960e052f071bfb62e19950e1f3acf...,09a0c14484d3f421e813bfd509b102be1d327aa1077aae...,https://cryptoginger29.wixsite.com/midnort,midnort-ada,MIDN,https://git.io/JKN8b,46.101.17.79
5506,72.86.45.91,3001,pool1k7gvq974d55yqcs4g6xyx4wy2fdlf97pffwgesjfq...,18295901333,1,NaN,92cf348da2c6a2a0666196009b4eb97101d3231955fd84...,b790c017d56d28406215468c4355c4525bf497c14a5c8c...,NaN,NaN,NaN,https://git.io/J6rex,72.86.45.91
5507,r1.jaguares.org,3001,pool1wz5klzzjstlzmqgv55q8zwaxfxvyjyjhq49u9hg3f...,10008229492,1,NaN,0915cb165e3532496e7183fce748827ac1f1aceebec819...,70a96f885282fe2d810ca500713ba64998491257054bc2...,NaN,NaN,NaN,https://git.io/J62wR,64.29.151.221


## The follwing code is used to run the IP Stack API and return the information on location and connection for the relays

In [58]:
def get_ip_data(df, outputfileName):
	new_df = pd.DataFrame()
	
	for i in range(len(df)):
		url = 'http://api.ipstack.com/'+df['resolved_ip'][i]+'?&fields=country_code,country_name,city,connection,region_code,region_name,zip,latitude,longitude'
		reqs = requests.get(url, params=payload).json()
		new_df = new_df.append(reqs, ignore_index=True)
		time.sleep(1)
	result = df.merge(new_df, left_index=True, right_index=True)
	
	return result.to_json(r''+outputfileName+'.json', orient='records') 
	return result.to_csv(r''+outputfileName+'.csv', index=False)

In [59]:
get_ip_data(hosts, 'test')

In [60]:
# df_all_relays_location = pd.read_json('all_relays_data.json')
df_test_locations = pd.read_json('test.json')


In [66]:
df_test_locations

,ip_host,port,pool_id,live_stake,delegators,description,hash,hex,homepage,name,ticker,url,resolved_ip,city,connection,country_code,country_name,latitude,longitude,region_code,region_name,zip,error,success,detail
0,54.220.20.40,3002.0,pool1z5uqdk7dzdxaae5633fqfcu2eqzy3a3rgtuvy087f...,49569650567271,3340,Octa's Performance Pool,ca7d12decf886e31f5226b5946c62edc81a7e40af95ce7...,153806dbcd134ddee69a8c5204e38ac80448f62342f8c2...,https://octaluso.dyndns.org,OctasPool,OCTAS,https://raw.githubusercontent.com/Octalus/card...,54.220.20.40,Dublin,"{'asn': 16509, 'isp': 'amazon.com, Inc'}",IE,Ireland,53.3539,-6.24333,L,Leinster,,,,
1,relays.mainnet.stakenuts.com,3001.0,pool1pu5jlj4q9w9jlxeu370a3c9myx47md5j5m2str0na...,19157905491154,165,StakeNuts.com,47c0c68cb57f4a5b4a87bad896fc274678e7aea98e200f...,0f292fcaa02b8b2f9b3c8f9fd8e0bb21abedb692a6d505...,https://stakenuts.com/,StakeNuts,NUTS,https://stakenuts.com/mainnet.json,199.247.23.219,Frankfurt am Main,"{'asn': 20473, 'isp': 'The Constant Company LLC'}",DE,Germany,50.1137,8.7119,HE,Hesse,60314,,,
2,relay.zenithpool.io,31400.0,pool1c8k78ny3xvsfgenhf4yzvpzwgzxmz0t0um0h2xnn2...,547697202732,9,"One owner, one operator, one pool. Operating s...",21cc44498ab6e74e081a805e629fff171a66d0c8d034f9...,c1ede3cc9133209466774d4826044e408db13d6fe6df75...,https://zenithpool.io,ZENITH,ZEN,https://metadata.zenithpool.io,193.122.12.254,Seattle,"{'asn': 31898, 'isp': 'Oracle Corporation'}",US,United States,47.611,-122.334,WA,Washington,98101,,,
3,95.183.53.20,6060.0,pool1q80jjs53w0fx836n8g38gtdwr8ck5zre3da90peux...,1862316910352,707,"Stability, Security, Reliability, Neutrality! ...",34fdde237812fab14d29a80423bb295f39122f4fea1aae...,01df29429173d263c7533a22742dae19f16a08798b7a57...,https://ispool.live/,Switzerland Investment,000,https://ispool.live/metadata,95.183.53.20,Zürich,"{'asn': 197988, 'isp': 'Solar Communications G...",CH,Switzerland,47.3742,8.53695,ZH,Zurich,8043,,,
4,95.217.222.194,6060.0,pool1q80jjs53w0fx836n8g38gtdwr8ck5zre3da90peux...,1862316910352,707,"Stability, Security, Reliability, Neutrality! ...",34fdde237812fab14d29a80423bb295f39122f4fea1aae...,01df29429173d263c7533a22742dae19f16a08798b7a57...,https://ispool.live/,Switzerland Investment,000,https://ispool.live/metadata,95.217.222.194,Tuusula,"{'asn': 24940, 'isp': 'Hetzner Online Gmbh'}",FI,Finland,60.4264,25.0113,18,Uusimaa,04300,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5504,137.184.178.161,6000,pool1pxsvz3yy606zr6qnhl2snvgzhcwny74pqaa2uvrk7...,130027614,1,"midnort staking pool, your nordic Cardano Stak...",cb738f574938eaa9e960e052f071bfb62e19950e1f3acf...,09a0c14484d3f421e813bfd509b102be1d327aa1077aae...,https://cryptoginger29.wixsite.com/midnort,midnort-ada,MIDN,https://git.io/JKN8b,137.184.178.161,Ashburn,{},US,United States,39.0437,-77.4742,VA,Virginia,20147,,,
5505,46.101.17.79,6000,pool1pxsvz3yy606zr6qnhl2snvgzhcwny74pqaa2uvrk7...,130027614,1,"midnort staking pool, your nordic Cardano Stak...",cb738f574938eaa9e960e052f071bfb62e19950e1f3acf...,09a0c14484d3f421e813bfd509b102be1d327aa1077aae...,https://cryptoginger29.wixsite.com/midnort,midnort-ada,MIDN,https://git.io/JKN8b,46.101.17.79,Blackheath,"{'asn': 14061, 'isp': 'Digitalocean LLC'}",GB,United Kingdom,51.5122,,ENG,England,EC2R,,,
5506,72.86.45.91,3001,pool1k7gvq974d55yqcs4g6xyx4wy2fdlf97pffwgesjfq...,18295901333,1,,92cf348da2c6a2a0666196009b4eb97101d3231955fd84...,b790c017d56d28406215468c4355c4525bf497c14a5c8c...,,,,https://git.io/J6rex,72.86.45.91,Ashburn,"{'asn': 701, 'isp': 'Mci Communications Servic...",US,United States,39.0396,-77.4816,VA,Virginia,20147,,,
5507,r1.jaguares.org,3001,pool1wz5klzzjstlzmqgv55q8zwaxfxvyjyjhq49u9hg3f...,10008229492,1,,0915cb165e3532496e7183fce748827ac1f1aceebec819...,70a96f885282fe2d810ca500713ba64998491257054bc2...,,,,https://git.io/J62wR,64.29.151.221,Rosemount,"{'asn': 30447, 'isp': 'internetnamesforbusines...",US,United States,45.0801,-93.1377,MN,Minnesota,55126,,,


In [65]:
df_test_locations.fillna('', inplace=True)
fig = px.scatter_geo(df_test_locations,
                     lat = 'latitude',
                     lon = 'longitude',
                #      locationmode='country names',
                #      locations="city",
                     color="pool_id", # which column to use to set the color of markers
                     hover_name="ticker", # column added to hover information
                     hover_data = ['live_stake', 'city', 'connection', 'delegators'],
                #      size="delegators", # size of markers
                     projection="natural earth")
fig.show()

In [36]:
final_df[final_df['ticker'] == 'PIADA']

,dns,dns_srv,ipv4,ipv6,port,live_stake,delegators,description,metadata_hash,pool_id_hex,homepage,name,pool_id,ticker,url,ip host,city_x,connection_x,country_code_x,country_name_x,latitude_x,longitude_x,region_code_x,region_name_x,zip_x,error_x,success_x,detail_x,ip_host,city_y,connection_y,country_code_y,country_name_y,latitude_y,longitude_y,region_code_y,region_name_y,zip_y,error_y,success_y,detail_y
3422,east.piada.io,,,,3000,207194064280,67,PIADA provides educational content specializin...,b89a6afcc92714b580d5449b7691e81b3b8ac417cc664e...,b8d8742c7b7b512468448429c776b3b0f824cef460db61...,https://tinyurl.com/3n6ftstn,𝛑 + ₳ = ∞,pool1hrv8gtrm0dgjg6zyss5uwa4nkruzfnh5vrdkr2say...,PIADA,https://piada.io/piada.metadata.json,209.145.58.238,San Marcos,"{'asn': 40021, 'isp': 'Contabo Inc.'}",US,United States,33.18708,-117.158478,CA,California,92069,None,NaN,None,209.145.58.238,San Marcos,"{'asn': 40021, 'isp': 'Contabo Inc.'}",US,United States,33.18708,-117.158478,CA,California,92069,None,NaN,None
3423,east.piada.io,,,,3002,207194064280,67,PIADA provides educational content specializin...,b89a6afcc92714b580d5449b7691e81b3b8ac417cc664e...,b8d8742c7b7b512468448429c776b3b0f824cef460db61...,https://tinyurl.com/3n6ftstn,𝛑 + ₳ = ∞,pool1hrv8gtrm0dgjg6zyss5uwa4nkruzfnh5vrdkr2say...,PIADA,https://piada.io/piada.metadata.json,143.198.206.176,Austin,"{'asn': 14061, 'isp': 'Digitalocean LLC'}",US,United States,30.26720,-97.742310,TX,Texas,78701,None,NaN,None,143.198.206.176,Austin,"{'asn': 14061, 'isp': 'Digitalocean LLC'}",US,United States,30.26720,-97.742310,TX,Texas,78701,None,NaN,None


Section  below may be used for cleaning...Don't upload to github.

In [26]:
final_df.to_csv('all_relays_data.csv', index = False)
final_df.to_json('all_relays_data.json', orient='records')